In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
from tqdm import trange
from src.svgd import SVGD
from src.gsvgd import FullGSVGDBatch
from src.kernel import RBF, BatchRBF
from src.manifold import Grassmann
import torch.autograd as autograd
from src.s_svgd import SlicedSVGD
from src.diffusion import Diffusion
from src.rand_mysvgd import mySVGD
import pickle
import argparse
import time
import numpyro
from numpyro.infer import MCMC, NUTS
import numpyro.distributions as npr_dist
import jax.random as random
import jax.numpy as jnp
import jax
from src.Tmy_svgd import etmySVGD

/home/zhoujk/anaconda3/envs/AG/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
parser = argparse.ArgumentParser(description="Running xshaped experiment.")
parser.add_argument("--dim", type=int, default=50, help="dimension")
parser.add_argument("--effdim", type=int, default=5, help="dimension")
parser.add_argument("--lr", type=float, default=0.0001, help="learning rate")
parser.add_argument("--lr_g", type=float, default=0.0001, help="learning rate for S-SVGD")
parser.add_argument(
    "--delta", type=float, default=0.1, help="stepsize for projections"
)
parser.add_argument(
    "--T", type=float, default=1e-4, help="noise multiplier for projections"
)
parser.add_argument("--nparticles", type=int, default=2500, help="no. of particles")
parser.add_argument("--epochs", type=int, default=20000, help="no. of epochs")
parser.add_argument("--noise", type=str, default="True", help="whether to add noise")
parser.add_argument("--kernel", type=str, default="rbf", help="kernel")
parser.add_argument("--gpu", type=int, default=6, help="gpu")
parser.add_argument("--seed", type=int, default=1, help="random seed")
parser.add_argument("--suffix", type=str, default="", help="suffix for res folder")
parser.add_argument("--method", type=str, default="svgd", help="svgd, gsvgd or s-svgd")
parser.add_argument("--save_every", type=int, default=100, help="batch size")

args = parser.parse_args([])

In [3]:
dim = args.dim
lr = args.lr
lr_g = args.lr_g
delta = args.delta
T = args.T
nparticles = args.nparticles
epochs = args.epochs
seed = args.seed
eff_dims = 10
add_noise = True if args.noise == "True" else False
save_every = args.save_every  # save metric values every 100 epochs
print(f"Running for lr: {lr}, nparticles: {nparticles}")

device = torch.device(f"cuda:{args.gpu}" if args.gpu != -1 else "cpu")


Running for lr: 0.0001, nparticles: 2500


In [4]:
results_folder = f"./res/diffusion{args.suffix}/{args.kernel}_epoch{epochs}_lr{lr}_delta{delta}_n{nparticles}_dim{dim}"
results_folder = f"{results_folder}/seed{seed}"

In [5]:
results_folder

'./res/diffusion/rbf_epoch20000_lr0.0001_delta0.1_n2500_dim50/seed1'

In [6]:
torch.__version__

'1.13.0'

In [7]:
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

if args.kernel == "rbf":
    Kernel = RBF
    BatchKernel = BatchRBF

In [8]:
print(f"Device: {device}")
torch.manual_seed(seed)
np.random.seed(seed)

## initialize conditional diffusion
beta = 5
sigma = 0.1

distribution = Diffusion(dim, beta, device=device)

loc = torch.arange(0, dim+1, 1)[1:]
distribution.loc = loc
noise_covariance = torch.diag(sigma**2 * torch.ones(len(loc), device=device))
distribution.set_noise_covariance(noise_covariance)
distribution.set_C()

x_true = distribution.brownian_motion((1, dim))
u_true = distribution.solution(x_true)

obs_noise = torch.normal(0, sigma, size=(1, len(loc))).to(device)
obs = u_true[:, loc] + obs_noise
distribution.obs = obs

# initialize particles
x0 = distribution.brownian_motion((nparticles, dim))

C = distribution.C.cpu().numpy()
dt = distribution.dt.cpu().numpy()
loc = distribution.loc.cpu().numpy()
beta = distribution.beta




Device: cuda:6


In [9]:
## SVGD
print("Running SVGD")
x = x0.clone().requires_grad_()
# sample from variational density
kernel = Kernel(method="med_heuristic")

Running SVGD


In [10]:
## SVGD
print("Running SVGD")
x = x0.clone().requires_grad_()
# sample from variational density
kernel = Kernel(method="med_heuristic")
svgd = SVGD(distribution, kernel, optim.Adam([x], lr=lr), device=device)

start = time.time()
svgd.fit(x, epochs=20000, save_every=save_every)
elapsed_time = time.time() - start

fitted_method = svgd
#particles = fitted_method.particles
particles = x

method_name = "svgd"


print(x)
## save results
pickle.dump(
     {
            **{"x_true": x_true},
            **{"u_true": u_true},
            **{"particles": particles},
            **{"time": elapsed_time}
    },
    open(results_folder + f"/particles_svgd2.p", "wb")
)

# target distribution
torch.save(distribution, results_folder + '/target_dist.p')
print(x.shape)

Running SVGD


100%|██████████| 20000/20000 [09:51<00:00, 33.84it/s]

tensor([[-0.0070,  0.2459,  0.2329,  ...,  0.1423,  0.1246,  0.1516],
        [-0.1544,  0.0652,  0.1707,  ...,  0.1015,  0.2012,  0.0487],
        [ 0.0425,  0.1803,  0.1499,  ...,  0.2066,  0.3006,  0.1842],
        ...,
        [-0.0333,  0.1570,  0.1721,  ..., -0.0695, -0.0295, -0.2599],
        [-0.0367,  0.1702,  0.1367,  ...,  0.2757,  0.3332,  0.3182],
        [-0.0623,  0.1435,  0.2078,  ...,  0.1374,  0.2315,  0.1624]],
       device='cuda:6', requires_grad=True)
torch.Size([2500, 50])


In [11]:
print("Running min_mySVGD")


x = x0.clone().requires_grad_()
# sample from variational density

lr = 0.001
theta = etmySVGD(kernel,device,distribution,  k = 2).update(x0,  n_iter = 20000,   lr= lr)
       

particles = theta

## save results
pickle.dump(
        {
            **{"x_true": x_true},
            **{"u_true": u_true},
            **{"particles": particles},
            **{"time": elapsed_time}
        },
open(results_folder + f"/particles_AUmp_svgd_2_test2.p", "wb")
)

print(particles)

Running min_mySVGD


100%|██████████| 20000/20000 [1:32:31<00:00,  3.60it/s]

tensor([[-0.0144,  0.2822,  0.2328,  ...,  0.1859,  0.1097,  0.1778],
        [-0.1199,  0.1213,  0.2185,  ...,  0.2523,  0.2674,  0.1023],
        [-0.0454,  0.1363,  0.3164,  ...,  0.1953,  0.2323,  0.1989],
        ...,
        [-0.1028,  0.1975,  0.2663,  ...,  0.1086,  0.2369, -0.0333],
        [-0.1007,  0.1955,  0.2054,  ...,  0.1031,  0.2451,  0.2946],
        [ 0.0084,  0.2738,  0.3174,  ...,  0.1744,  0.1747,  0.1167]],
       device='cuda:6')


In [12]:
print("Running min_mySVGD")


x = x0.clone().requires_grad_()
# sample from variational density

lr = 0.001
theta = etmySVGD(kernel,device,distribution,  k = 5).update(x0,  n_iter = 20000,   lr= lr)
       

particles = theta

## save results
pickle.dump(
        {
            **{"x_true": x_true},
            **{"u_true": u_true},
            **{"particles": particles},
            **{"time": elapsed_time}
        },
open(results_folder + f"/particles_AUmp_svgd_5_test2.p", "wb")
)

print(particles)

Running min_mySVGD


100%|██████████| 20000/20000 [1:30:58<00:00,  3.66it/s]

tensor([[-0.0722,  0.1934,  0.2799,  ...,  0.1995,  0.1243,  0.1845],
        [-0.1747,  0.0318,  0.0816,  ...,  0.2371,  0.3574,  0.1090],
        [-0.0167,  0.2950,  0.3230,  ...,  0.2084,  0.2703,  0.2057],
        ...,
        [-0.2138, -0.0368,  0.0357,  ...,  0.0760,  0.2491, -0.0299],
        [ 0.0410,  0.2147,  0.1463,  ...,  0.1175,  0.2231,  0.2946],
        [-0.0825,  0.1947,  0.1941,  ...,  0.2033,  0.1883,  0.1167]],
       device='cuda:6')


In [13]:
print("Running min_mySVGD")


x = x0.clone().requires_grad_()
# sample from variational density

lr = 0.001
theta = etmySVGD(kernel,device,distribution,  k = 10).update(x0,  n_iter = 20000,   lr= lr)
       

particles = theta

## save results
pickle.dump(
        {
            **{"x_true": x_true},
            **{"u_true": u_true},
            **{"particles": particles},
            **{"time": elapsed_time}
        },
open(results_folder + f"/particles_AUmp_svgd_10_test2.p", "wb")
)

print(particles)

Running min_mySVGD


100%|██████████| 20000/20000 [1:32:52<00:00,  3.59it/s]

tensor([[ 0.1026,  0.3220,  0.3208,  ...,  0.1987,  0.1184,  0.1879],
        [-0.0293,  0.2101,  0.2261,  ...,  0.2784,  0.3946,  0.1157],
        [ 0.0502,  0.3053,  0.3626,  ...,  0.1686,  0.2779,  0.2090],
        ...,
        [-0.1142,  0.1117,  0.2137,  ...,  0.0786,  0.1657, -0.0266],
        [-0.0567,  0.2132,  0.2555,  ...,  0.1569,  0.2174,  0.2946],
        [ 0.0134,  0.2328,  0.2602,  ...,  0.1766,  0.1821,  0.1235]],
       device='cuda:6')


In [14]:
## S-SVGD
# sample from variational density
print("Running S-SVGD")
x_s_svgd = x0.clone().requires_grad_()
s_svgd = SlicedSVGD(distribution, device=device)

start = time.time()
x_s_svgd, metric_s_svgd = s_svgd.fit(
    samples=x_s_svgd, 
    n_epoch=epochs, 
    lr=lr_g,
    eps=lr,
    save_every=save_every
)
elapsed_time = time.time() - start

fitted_method = s_svgd
particles = x_s_svgd

pickle.dump(
{
            **{"x_true": x_true},
            **{"u_true": u_true},
            **{"particles": particles},
            **{"time": elapsed_time}
        },
open(results_folder + f"/particles_ssvgd_test2.p", "wb")
)

print(particles)

Running S-SVGD


100%|██████████| 20000/20000 [2:15:07<00:00,  2.47it/s]  

tensor([[ 0.2159,  0.4214,  0.3229,  ...,  0.1981,  0.1173,  0.1986],
        [-0.0080,  0.2814,  0.2784,  ...,  0.2696,  0.3969,  0.1141],
        [ 0.1020,  0.4343,  0.5375,  ...,  0.1610,  0.2609,  0.1939],
        ...,
        [-0.1681,  0.0330,  0.2510,  ...,  0.0689,  0.1555, -0.0336],
        [-0.0944,  0.2418,  0.3511,  ...,  0.1478,  0.2178,  0.2985],
        [ 0.0530,  0.2160,  0.2954,  ...,  0.1756,  0.1702,  0.1219]],
       device='cuda:6', grad_fn=<CloneBackward0>)
